In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Test DFA Minimization

In this notebook, we verify our minimization algorithm using a concrete example.
We manually construct a DFA, visualize it, run the minimization, and visualize the result.

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, Tuple } from "recursive-set";


import { minimize, computeMinimizationPartition, DFA, TransRelDet, DFAState, State, Partition } from "./07-Minimize";
import { dfa2string, dfa2dot } from "./FSM-2-Dot";
import { key } from "./05-DFA-2-RegExp";

const viz = await instance();

### Defining the DFA

We define the DFA $A = (Q, \Sigma, \delta, q_0, F)$ manually.

**Implementation Details:**
* **State Identity:** To ensure correct behavior with `RecursiveSet` (which relies on hashing and structural equality), we pre-create the singleton sets representing our states ($\{0\}, \{1\}, \dots, \{8\}$). This allows us to cache them and ensures that every reference to state $i$ uses the exact same object.
* **Bulk Creation:** We use `RecursiveSet.fromSortedUnsafe` and `fromArray` for initialization. This is significantly more performant than adding elements one by one in a loop, as it bypasses redundant sorting checks.

**Transitions:**
The transitions are defined as follows:
* $(0, a) \to 1, \quad (0, b) \to 6$
* $(1, a) \to 2, \quad (1, b) \to 7$
* ... (see code for full definition)

In [ ]:
// Pre-create ALL state instances (Singletons {0}..{8})
const states: DFAState[] = Array.from({ length: 9 }, (_, i) => 
    RecursiveSet.fromSortedUnsafe([i])
);

// Helper: Access the cached instance by index
const S = (i: number) => states[i];

// The alphabet
const Sigma = new RecursiveSet<string>("a", "b");

// The Transition Function
const delta = new Map<string, DFAState>();

const addTrans = (from: number, char: string, to: number) => {
  delta.set(key(S(from), char), S(to));
};

// Define transitions
addTrans(0, "a", 1); addTrans(0, "b", 6);
addTrans(1, "a", 2); addTrans(1, "b", 7);
addTrans(2, "a", 3); addTrans(2, "b", 6);
addTrans(3, "a", 0); addTrans(3, "b", 7);
addTrans(4, "a", 5); addTrans(4, "b", 2);
addTrans(5, "a", 6); addTrans(5, "b", 8);
addTrans(6, "a", 7); addTrans(6, "b", 2);
addTrans(7, "a", 0); addTrans(7, "b", 3);
addTrans(8, "a", 4); addTrans(8, "b", 8);

// Set of all states Q
const Q = RecursiveSet.fromArray(states);

// Accepting States A = {2, 3, 6, 7}
const A_states = RecursiveSet.fromArray([S(2), S(3), S(6), S(7)]);

// The Start State q0
const q0 = S(0);

// Construct the DFA object
const A: DFA = {
  Q: Q,
  Sigma: Sigma,
  delta: delta,
  q0: q0,
  A: A_states
};

### Visualize Original DFA

We generate both the text representation and the Graphviz diagram for the constructed automaton.

**Interpreting the Output:**
The graph nodes are labeled automatically with generated names ($S_0, S_1, \dots$). To understand which actual states these correspond to (e.g., does $S_0$ correspond to $\{0\}$ or $\{4\}$?), refer to the **"state encoding"** section in the text output above the graph.

In [ ]:
console.log(dfa2string(A));

const { dot: dot1 } = dfa2dot(A);
display.html(viz.renderString(dot1, { format: "svg" }));

## 1. Analyze Equivalence Classes

Before constructing the final minimized DFA, we inspect the intermediate result: the **Partition** of the state space.



The function `computeMinimizationPartition` groups states that cannot be distinguished by any input string. The result is a set of sets (e.g., $\{\{2, 6\}, \{1\}\}$).
* **Singleton Sets:** States that are unique in their behavior (e.g., `{0}`).
* **Merged Sets:** States that are equivalent and will be merged into a single state in the minimal DFA (e.g., `{2}` and `{6}`).

In [ ]:
const partition : Partition = computeMinimizationPartition(A);

In [ ]:
console.log("Found Equivalence Classes (Merged States):");
for (const cls of partition) {
    console.log(cls); 
}

## 2. Construct and Visualize Minimized DFA

Now we run the full `minimize` function to generate the final automaton.

### Traceability via Union Strategy
The `minimize` function applies a **Union Strategy** to normalize the resulting states:
It constructs the new state for an equivalence class by forming the **Union** of all original states belonging to that class.

**Example:**
If states $\{2\}$ and $\{6\}$ are equivalent, the resulting minimized state will be $\{2, 6\}$.

This allows us to trace the origin of the minimized states directly in the visual output below, as the state encoding (e.g., $S_2 = \{2, 6\}$) explicitly lists the merged states.

In [ ]:
const F : DFA = minimize(A);

In [ ]:
console.log("Minimized DFA Structure:");
console.log(dfa2string(F));

const { dot: dot2 } = dfa2dot(F);
display.html(viz.renderString(dot2, { format: "svg" }));